In [1]:
import pymysql
import pandas as pd
from tqdm import tqdm

In [2]:
conn = pymysql.connect(user='root', password='', database='sql_final')

In [3]:
cursor = conn.cursor()

cursor.execute("SHOW TABLES")

records = cursor.fetchall()



In [4]:
print(records)

(('accident',), ('cause',), ('condi',), ('date',), ('handle',), ('people',))


In [5]:
df = pd.read_csv('./NPA_TMA1_Need.csv')


In [6]:
df

,事故類型及型態子類別名稱,發生年度,發生月份,發生日期,處理單位名稱警局層,發生地點,天候名稱,道路類別-第1當事者-名稱,當事者順位,當事者區分-類別-大類別名稱-車種,當事者屬-性-別名稱,當事者事故發生時年齡,肇因研判子類別名稱-主要,肇事逃逸類別名稱-是否肇逃
0,撞號誌、標誌桿,2022,1.0,20220101.0,雲林縣警察局,雲林縣虎尾鎮雲145公路路燈290570號,晴,市區道路,1.0,機車,男,17.0,未注意車前狀態,否
1,撞號誌、標誌桿,2022,1.0,20220101.0,雲林縣警察局,雲林縣虎尾鎮雲145公路路燈290570號,晴,市區道路,2.0,NaN,無或物(動物、堆置物),-1.0,未注意車前狀態,否
2,追撞,2022,1.0,20220101.0,國道公路警察局,屏東縣九如鄉國道3號 393公里600.0公尺處南側向外側,晴,國道,1.0,小客車,男,34.0,未注意車前狀態,否
3,追撞,2022,1.0,20220101.0,國道公路警察局,屏東縣九如鄉國道3號 393公里600.0公尺處南側向外側,晴,國道,2.0,小貨車(含客、貨兩用),男,43.0,未注意車前狀態,否
4,追撞,2022,1.0,20220101.0,國道公路警察局,屏東縣九如鄉國道3號 393公里600.0公尺處南側向外側,晴,國道,3.0,NaN,無或物(動物、堆置物),-1.0,未注意車前狀態,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4420,穿越道路中,2022,12.0,20221231.0,基隆市警察局,基隆市仁愛區仁一路 / 基隆市仁愛區愛四路,雨,市區道路,2.0,機車,男,23.0,未依標誌、標線、號誌或手勢指揮穿越道路,否
4421,穿越道路中,2022,12.0,20221231.0,基隆市警察局,基隆市仁愛區仁一路 / 基隆市仁愛區愛四路,雨,市區道路,3.0,人,男,70.0,未依標誌、標線、號誌或手勢指揮穿越道路,否
4422,其他,2022,12.0,20221231.0,基隆市警察局,基隆市七堵區基隆市七堵區明德二路工東高幹35B8852CC38,雨,市區道路,1.0,小貨車(含客、貨兩用),男,40.0,開啟車門不當而肇事,否
4423,其他,2022,12.0,20221231.0,基隆市警察局,基隆市七堵區基隆市七堵區明德二路工東高幹35B8852CC38,雨,市區道路,2.0,機車,男,64.0,開啟車門不當而肇事,否


In [7]:
import datetime
from datetime import datetime
s = 20220701.0
s = str(int(s))
day = datetime.strptime(s, '%Y%m%d')
day_only=day.date()
print(type(day_only))
print(day_only)

<class 'datetime.date'>
2022-07-01


In [8]:
import datetime
from datetime import datetime

for i, row in df.iterrows() :
    case_date = str(int(row['發生日期']))
    year = int(row['發生年度'])
    month = int(row['發生月份'])
    day = int(case_date[-2:])
    
    case_date = datetime.strptime(case_date, '%Y%m%d')
    case_date_only=case_date.date()

    data = (case_date_only, year, month, day)
    query = "SELECT * FROM date WHERE datetime = %s "
    cursor.execute(query, (case_date_only))
    result = cursor.fetchone()

    # 如果資料不存在，則插入資料
    if not result:
        query = "INSERT INTO date (datetime, year, month, day) VALUES (%s, %s, %s, %s)"
        print(data)
        cursor.execute(query, data)
        print("insert:",case_date_only, year, month, day)

# 提交事務
conn.commit()

In [9]:
for i, row in df.iterrows() :
    weather = str(row['天候名稱'])
    road_type = str(row['道路類別-第1當事者-名稱'])
    
    data= (weather,road_type)
    
    query = "SELECT * FROM condi WHERE (weather, road_type) = (%s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    
    if not result:
        print(data)
        query = "INSERT INTO condi (weather, road_type) VALUES (%s, %s)"
        cursor.execute(query, data)
        print("insert:",weather, road_type)

# 提交事務
conn.commit()

('晴', '市區道路')
insert: 晴 市區道路
('晴', '國道')
insert: 晴 國道
('晴', '省道')
insert: 晴 省道
('晴', '其他')
insert: 晴 其他
('晴', '縣道')
insert: 晴 縣道
('晴', '村里道路')
insert: 晴 村里道路
('陰', '市區道路')
insert: 陰 市區道路
('雨', '省道')
insert: 雨 省道
('雨', '市區道路')
insert: 雨 市區道路
('陰', '其他')
insert: 陰 其他
('陰', '省道')
insert: 陰 省道
('晴', '鄉道')
insert: 晴 鄉道
('雨', '縣道')
insert: 雨 縣道
('陰', '村里道路')
insert: 陰 村里道路
('雨', '村里道路')
insert: 雨 村里道路
('霧或煙', '市區道路')
insert: 霧或煙 市區道路
('霧或煙', '國道')
insert: 霧或煙 國道
('雨', '其他')
insert: 雨 其他
('陰', '鄉道')
insert: 陰 鄉道
('霧或煙', '縣道')
insert: 霧或煙 縣道
('雨', '國道')
insert: 雨 國道
('陰', '專用道路')
insert: 陰 專用道路
('雨', '鄉道')
insert: 雨 鄉道
('陰', '縣道')
insert: 陰 縣道
('陰', '國道')
insert: 陰 國道
('暴雨', '市區道路')
insert: 暴雨 市區道路
('強風', '省道')
insert: 強風 省道
('晴', '專用道路')
insert: 晴 專用道路
('暴雨', '省道')
insert: 暴雨 省道


In [10]:
for i, row in df.iterrows() :
    cause_type = str(row['肇因研判子類別名稱-主要'])
    hit_and_run = str(row['肇事逃逸類別名稱-是否肇逃'])
    
    if hit_and_run == "否":
        hit_and_run=0
    elif hit_and_run == "是":
        hit_and_run=1
    
    data= (cause_type, hit_and_run)
    
    query = "SELECT * FROM cause WHERE (cause_type, hit_and_run) = (%s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    
    if not result:
        print(data)
        query = "INSERT INTO cause (cause_type, hit_and_run) VALUES (%s, %s)"
        cursor.execute(query, data)
        print("insert:",cause_type, hit_and_run)

# 提交事務
conn.commit()

('未注意車前狀態', 0)
insert: 未注意車前狀態 0
('不明原因肇事', 0)
insert: 不明原因肇事 0
('其他引起事故之疏失或行為', 0)
insert: 其他引起事故之疏失或行為 0
('違反號誌管制或指揮', 0)
insert: 違反號誌管制或指揮 0
('未依規定讓車', 0)
insert: 未依規定讓車 0
('未保持行車安全間隔', 0)
insert: 未保持行車安全間隔 0
('開啟車門不當而肇事', 0)
insert: 開啟車門不當而肇事 0
('未依標誌、標線、號誌或手勢指揮穿越道路', 0)
insert: 未依標誌、標線、號誌或手勢指揮穿越道路 0
('起步未注意其他車(人)安全', 0)
insert: 起步未注意其他車(人)安全 0
('酒醉(後)駕駛失控', 0)
insert: 酒醉(後)駕駛失控 0
('其他引起事故之違規或不當行為', 0)
insert: 其他引起事故之違規或不當行為 0
('未依規定行走行人穿越道、地下道、天橋而穿越道路', 0)
insert: 未依規定行走行人穿越道、地下道、天橋而穿越道路 0
('變換車道或方向不當', 0)
insert: 變換車道或方向不當 0
('倒車未依規定', 0)
insert: 倒車未依規定 0
('未依規定讓車', 1)
insert: 未依規定讓車 1
('未注意車前狀態', 1)
insert: 未注意車前狀態 1
('超速失控', 1)
insert: 超速失控 1
('超速失控', 0)
insert: 超速失控 0
('違規超車', 0)
insert: 違規超車 0
('車輪脫落或輪胎爆裂', 0)
insert: 車輪脫落或輪胎爆裂 0
('左轉彎未依規定', 0)
insert: 左轉彎未依規定 0
('違反特定標誌(線)禁制', 0)
insert: 違反特定標誌(線)禁制 0
('右轉彎未依規定', 0)
insert: 右轉彎未依規定 0
('未依規定減速', 0)
insert: 未依規定減速 0
('未保持行車安全間隔', 1)
insert: 未保持行車安全間隔 1
('橫越道路不慎', 0)
insert: 橫越道路不慎 0
('違反號誌管制或指揮', 1)
insert: 違反號誌管制或指揮 1
('在道路上嬉

In [11]:
for i, row in df.iterrows() :
    city = str(row['發生地點'])
    station = str(row['處理單位名稱警局層'])
    city=city[0:3]
    
    data= (city, station)
    
    query = "SELECT * FROM handle WHERE (city, station) = (%s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    
    if not result:
        print(data)
        query = "INSERT INTO handle (city, station) VALUES (%s, %s)"
        cursor.execute(query, data)
        print("insert:", city, station)

# 提交事務
conn.commit()

('雲林縣', '雲林縣警察局')
insert: 雲林縣 雲林縣警察局
('屏東縣', '國道公路警察局')
insert: 屏東縣 國道公路警察局
('宜蘭縣', '宜蘭縣政府警察局')
insert: 宜蘭縣 宜蘭縣政府警察局
('南投縣', '南投縣政府警察局')
insert: 南投縣 南投縣政府警察局
('屏東縣', '屏東縣政府警察局')
insert: 屏東縣 屏東縣政府警察局
('嘉義縣', '嘉義縣警察局')
insert: 嘉義縣 嘉義縣警察局
('新竹縣', '新竹縣政府警察局')
insert: 新竹縣 新竹縣政府警察局
('桃園市', '桃園市政府警察局')
insert: 桃園市 桃園市政府警察局
('新竹市', '新竹市警察局')
insert: 新竹市 新竹市警察局
('臺南市', '臺南市政府警察局')
insert: 臺南市 臺南市政府警察局
('高雄市', '高雄市政府警察局')
insert: 高雄市 高雄市政府警察局
('臺北市', '臺北市政府警察局')
insert: 臺北市 臺北市政府警察局
('臺中市', '臺中市政府警察局')
insert: 臺中市 臺中市政府警察局
('彰化縣', '彰化縣警察局')
insert: 彰化縣 彰化縣警察局
('新北市', '新北市政府警察局')
insert: 新北市 新北市政府警察局
('臺南市', '國道公路警察局')
insert: 臺南市 國道公路警察局
('臺東縣', '臺東縣警察局')
insert: 臺東縣 臺東縣警察局
('金門縣', '金門縣警察局')
insert: 金門縣 金門縣警察局
('苗栗縣', '國道公路警察局')
insert: 苗栗縣 國道公路警察局
('苗栗縣', '苗栗縣警察局')
insert: 苗栗縣 苗栗縣警察局
('高雄市', '國道公路警察局')
insert: 高雄市 國道公路警察局
('澎湖縣', '澎湖縣政府警察局')
insert: 澎湖縣 澎湖縣政府警察局
('花蓮縣', '花蓮縣警察局')
insert: 花蓮縣 花蓮縣警察局
('臺南市', '鐵路警察局')
insert: 臺南市 鐵路警察局
('臺中市', '國道公路警察局')
insert: 臺中市 國道公路警察局
('桃園市', '國道公路警察局')
inse

In [12]:
def getdateid( datetime ):
    d = str(int(datetime))
    query = "SELECT * FROM date WHERE datetime = %s"
    cursor.execute(query, d)
    result = cursor.fetchone()
    return result[0]

In [13]:
def getcondiid( weather, road_type ):
    data= (str(weather),str(road_type))
    
    query = "SELECT * FROM condi WHERE (weather, road_type) = (%s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    return result[0]

In [14]:
def getcauseid( cause, hit_and_run ):
    if hit_and_run == "否":
        hit_and_run=0
    elif hit_and_run == "是":
        hit_and_run=1
    data= (str(cause),int(hit_and_run))
    
    query = "SELECT * FROM cause WHERE (cause_type, hit_and_run) = (%s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    return result[0]

In [15]:
def gethandleid( city, station ):
    data= (str(city),str(station))
    
    query = "SELECT * FROM handle WHERE (city, station) = (%s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    return result[0]

In [16]:
gethandleid('臺北市', '國道公路警察局')

28

In [17]:
for i, row in tqdm(df.iterrows()) :
    date_id = getdateid(row['發生日期'])
    cause_id = getcauseid(row['肇因研判子類別名稱-主要'],row['肇事逃逸類別名稱-是否肇逃'])
    condi_id =getcondiid(row['天候名稱'],row['道路類別-第1當事者-名稱'])
    handle_id = gethandleid(str(row['發生地點'][:3]), row['處理單位名稱警局層'])
    accident_type = row['事故類型及型態子類別名稱']
    
    data = (accident_type, date_id, handle_id,cause_id,condi_id)
    #print(data)
    
    
    query = "SELECT * FROM accident WHERE (accident_type, date_id, handle_id, cause_id, condition_id ) = (%s, %s, %s, %s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    
    if not result:
        #print(data)
        query = "INSERT INTO accident (accident_type, date_id, handle_id, cause_id, condition_id ) VALUES (%s, %s, %s, %s, %s)"
        cursor.execute(query, data)
        print("insert:", data)

# 提交事務
conn.commit()

51it [00:00, 504.72it/s]

insert: ('撞號誌、標誌桿', datetime.date(2022, 1, 1), 1, 1, 1)
insert: ('追撞', datetime.date(2022, 1, 1), 2, 1, 2)
insert: ('衝出路外', datetime.date(2022, 1, 1), 3, 2, 3)
insert: ('衝出路外', datetime.date(2022, 1, 1), 4, 1, 3)
insert: ('其他', datetime.date(2022, 1, 1), 5, 3, 1)
insert: ('其他', datetime.date(2022, 1, 2), 3, 4, 4)
insert: ('同向通行中', datetime.date(2022, 1, 2), 6, 1, 5)
insert: ('側撞', datetime.date(2022, 1, 2), 7, 5, 3)
insert: ('其他', datetime.date(2022, 1, 2), 8, 1, 3)
insert: ('同向通行中', datetime.date(2022, 1, 3), 9, 6, 1)
insert: ('側撞', datetime.date(2022, 1, 3), 10, 5, 1)
insert: ('側撞', datetime.date(2022, 1, 3), 1, 5, 5)
insert: ('撞路樹、電桿', datetime.date(2022, 1, 3), 8, 1, 1)
insert: ('路口交岔撞', datetime.date(2022, 1, 3), 11, 5, 1)
insert: ('其他', datetime.date(2022, 1, 3), 12, 7, 1)
insert: ('路上翻車、摔倒', datetime.date(2022, 1, 3), 6, 1, 3)
insert: ('穿越道路中', datetime.date(2022, 1, 4), 12, 8, 1)
insert: ('側撞', datetime.date(2022, 1, 4), 10, 9, 1)
insert: ('穿越道路中', datetime.date(2022, 1, 4), 7,

149it [00:00, 490.37it/s]

insert: ('側撞', datetime.date(2022, 1, 9), 14, 5, 6)
insert: ('側撞', datetime.date(2022, 1, 9), 14, 15, 6)
insert: ('其他', datetime.date(2022, 1, 9), 8, 1, 7)
insert: ('其他', datetime.date(2022, 1, 9), 8, 16, 7)
insert: ('衝出路外', datetime.date(2022, 1, 9), 14, 2, 6)
insert: ('其他', datetime.date(2022, 1, 9), 15, 4, 9)
insert: ('其他', datetime.date(2022, 1, 10), 19, 3, 2)
insert: ('穿越道路中', datetime.date(2022, 1, 10), 8, 17, 1)
insert: ('穿越道路中', datetime.date(2022, 1, 10), 8, 18, 1)
insert: ('其他', datetime.date(2022, 1, 10), 20, 19, 10)
insert: ('其他', datetime.date(2022, 1, 10), 21, 20, 2)
insert: ('側撞', datetime.date(2022, 1, 10), 8, 21, 1)
insert: ('對向通行中', datetime.date(2022, 1, 10), 22, 1, 7)
insert: ('撞路樹、電桿', datetime.date(2022, 1, 10), 11, 10, 1)
insert: ('佇立路邊(外)', datetime.date(2022, 1, 11), 13, 3, 1)
insert: ('同向通行中', datetime.date(2022, 1, 11), 15, 1, 9)
insert: ('其他', datetime.date(2022, 1, 11), 12, 22, 9)
insert: ('同向擦撞', datetime.date(2022, 1, 11), 11, 1, 1)
insert: ('撞橋樑、建築物', da

216it [00:00, 386.56it/s]

...

 ('同向擦撞', datetime.date(2022, 12, 11), 13, 6, 1)
insert: ('側撞', datetime.date(2022, 12, 11), 5, 4, 3)
insert: ('其他', datetime.date(2022, 12, 12), 3, 33, 15)
insert: ('其他', datetime.date(2022, 12, 12), 3, 2, 15)
insert: ('側撞', datetime.date(2022, 12, 12), 13, 22, 1)
insert: ('側撞', datetime.date(2022, 12, 12), 15, 4, 9)
insert: ('其他', datetime.date(2022, 12, 12), 26, 52, 25)
insert: ('對向擦撞', datetime.date(2022, 12, 12), 20, 13, 5)
insert: ('撞路樹、電桿', datetime.date(2022, 12, 13), 1, 1, 6)
insert: ('其他', datetime.date(2022, 12, 14), 10, 1, 1)
insert: ('其他', datetime.date(2022, 12, 14), 14, 24, 6)
insert: ('路口交岔撞', datetime.date(2022, 12, 14), 1, 24, 19)
insert: ('同向擦撞', datetime.date(2022, 12, 15), 21, 1, 4)
insert: ('側撞', datetime.date(2022, 12, 15), 20, 4, 8)
insert: ('撞路樹、電桿', datetime.date(2022, 12, 15), 9, 2, 9)
insert: ('撞護欄(樁)', datetime.date(2022, 12, 15), 17, 1, 3)
insert: ('同向通行中', datetime.date(2022, 12, 15), 14, 6, 6)
insert: ('撞路樹、電桿', datetime.date(2022, 12, 16), 10, 2, 1)
ins

4401it [00:12, 381.25it/s]

insert: ('同向擦撞', datetime.date(2022, 12, 20), 15, 6, 1)
insert: ('追撞', datetime.date(2022, 12, 20), 26, 1, 2)
insert: ('路口交岔撞', datetime.date(2022, 12, 20), 32, 1, 1)
insert: ('路口交岔撞', datetime.date(2022, 12, 20), 10, 5, 1)
insert: ('側撞', datetime.date(2022, 12, 20), 10, 9, 1)
insert: ('其他', datetime.date(2022, 12, 21), 15, 2, 7)
insert: ('路口交岔撞', datetime.date(2022, 12, 21), 13, 5, 1)
insert: ('撞護欄(樁)', datetime.date(2022, 12, 22), 20, 10, 6)
insert: ('側撞', datetime.date(2022, 12, 22), 10, 5, 1)
insert: ('同向擦撞', datetime.date(2022, 12, 23), 20, 1, 5)
insert: ('其他', datetime.date(2022, 12, 23), 1, 1, 1)
insert: ('同向通行中', datetime.date(2022, 12, 24), 13, 1, 1)
insert: ('側撞', datetime.date(2022, 12, 24), 10, 21, 1)
insert: ('路上翻車、摔倒', datetime.date(2022, 12, 24), 15, 10, 1)
insert: ('穿越道路中', datetime.date(2022, 12, 25), 20, 1, 1)
insert: ('對撞', datetime.date(2022, 12, 26), 6, 22, 5)
insert: ('同向擦撞', datetime.date(2022, 12, 26), 11, 25, 1)
insert: ('同向擦撞', datetime.date(2022, 12, 26), 11,

4425it [00:12, 357.85it/s]

 ('撞路樹、電桿', datetime.date(2022, 12, 27), 20, 41, 5)
insert: ('穿越道路中', datetime.date(2022, 12, 27), 13, 37, 1)
insert: ('側撞', datetime.date(2022, 12, 28), 1, 9, 4)
insert: ('撞路樹、電桿', datetime.date(2022, 12, 29), 11, 1, 1)
insert: ('同向擦撞', datetime.date(2022, 12, 30), 11, 13, 1)
insert: ('穿越道路中', datetime.date(2022, 12, 31), 27, 8, 9)
insert: ('其他', datetime.date(2022, 12, 31), 27, 7, 9)


In [18]:
def geteventid( row ):
    date_id = getdateid(row['發生日期'])
    cause_id = getcauseid(row['肇因研判子類別名稱-主要'],row['肇事逃逸類別名稱-是否肇逃'])
    condi_id =getcondiid(row['天候名稱'],row['道路類別-第1當事者-名稱'])
    handle_id = gethandleid(str(row['發生地點'][:3]), row['處理單位名稱警局層'])
    accident_type = row['事故類型及型態子類別名稱']
    data = (accident_type, date_id, handle_id,cause_id,condi_id)
    query = "SELECT * FROM accident WHERE (accident_type, date_id, handle_id, cause_id, condition_id ) = (%s, %s, %s, %s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    return result[0]

In [19]:
for i, row in tqdm(df.iterrows()) :
    car = True
    age = int(row['當事者事故發生時年齡'])
    car_type = str(row['當事者區分-類別-大類別名稱-車種'])
    if car_type == '小貨車(含客、貨兩用)':
        car_type = '小貨車'
    if pd.isnull(row['當事者區分-類別-大類別名稱-車種']) :
        car =False
    event_id = geteventid(row)
    people_order = int(row['當事者順位'])
    sex = str(row['當事者屬-性-別名稱'])
    if sex == '無或物(動物、堆置物)':
        sex = '無或物'
    elif sex == '肇事逃逸尚未查獲':
        sex = '未知'

    if car :
        data = (age, car_type, event_id, people_order, sex)
    else :
        data = (age, event_id, people_order, sex)

    if car :
        query = "SELECT * FROM people WHERE (age, car_type, event_id, people_order, sex) = (%s, %s, %s, %s, %s)"
    else :
        query = "SELECT * FROM people WHERE (age, event_id, people_order, sex) = (%s, %s, %s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()

    if not result:
        #print(data)
        if car :
            query = "INSERT INTO people (age, car_type, event_id, people_order, sex) VALUES (%s, %s, %s, %s, %s)"
        else :
            query = "INSERT INTO people (age, event_id, people_order, sex) VALUES (%s, %s, %s, %s)"
        cursor.execute(query, data)
        print("insert:", data)
# 提交事務
conn.commit()

26it [00:00, 257.62it/s]

insert: (17, '機車', 1, 1, '男')
insert: (-1, 1, 2, '無或物')
insert: (34, '小客車', 2, 1, '男')
insert: (43, '小貨車', 2, 2, '男')
insert: (-1, 2, 3, '無或物')
insert: (56, '小客車', 2, 4, '女')
insert: (24, '機車', 3, 1, '男')
insert: (-1, 3, 2, '無或物')
insert: (18, '機車', 4, 1, '女')
insert: (-1, 4, 2, '無或物')
insert: (19, '人', 4, 3, '男')
insert: (68, '人', 5, 1, '男')
insert: (22, '小客車', 5, 2, '女')
insert: (20, '機車', 6, 1, '男')
insert: (42, '小客車', 6, 2, '男')
insert: (67, '小貨車', 7, 1, '男')
insert: (81, '人', 7, 2, '女')
insert: (47, '小客車', 8, 1, '男')
insert: (29, '機車', 8, 2, '男')
insert: (48, '機車', 9, 1, '男')
insert: (56, '小貨車', 9, 2, '男')
insert: (67, '大貨車', 10, 1, '男')
insert: (88, '人', 10, 2, '男')
insert: (38, '曳引車', 11, 1, '男')
insert: (86, '機車', 11, 2, '女')
insert: (49, '曳引車', 12, 1, '男')
insert: (24, '小客車', 12, 2, '男')
insert: (29, '人', 12, 3, '女')
insert: (38, '人', 12, 4, '男')
insert: (50, '機車', 13, 1, '男')
insert: (-1, 13, 2, '無或物')
insert: (52, '機車', 14, 1, '女')
insert: (78, '機車', 14, 2, '男')
insert: (44,

53it [00:00, 257.70it/s]

 (0, 21, 3, '無或物')
insert: (65, '機車', 21, 4, '女')
insert: (0, 21, 5, '無或物')
insert: (0, 21, 6, '無或物')
insert: (71, '人', 21, 7, '女')
insert: (67, '人', 21, 8, '女')
insert: (68, '機車', 22, 1, '女')
insert: (28, '機車', 22, 2, '女')
insert: (52, '機車', 23, 1, '女')
insert: (59, '半聯結車', 23, 2, '男')
insert: (67, '人', 23, 3, '女')
insert: (25, '小客車', 24, 1, '男')
insert: (86, '人', 24, 2, '男')
insert: (30, '人', 24, 3, '女')
insert: (43, '小客車', 25, 1, '女')
insert: (18, '機車', 25, 2, '男')
insert: (43, '小客車', 26, 1, '男')
insert: (81, '機車', 26, 2, '男')
insert: (73, '人', 27, 1, '女')
insert: (32, '小客車', 27, 2, '男')
insert: (43, '小客車', 28, 1, '男')
insert: (0, 28, 2, '無或物')
insert:

78it [00:00, 252.46it/s]

...

insert: (24, '全聯結車', 1762, 1, '男')
insert: (86, '慢車', 1762, 2, '男')
insert: (60, '小貨車', 1763, 1, '男')
insert: (44, '小貨車', 1763, 2, '男')
insert: (44, '小貨車', 1763, 3, '男')
insert: (47, '機車', 1763, 4, '女')
insert: (72, '機車', 1763, 5, '男')
insert: (44, '機車', 1763, 6, '女')
insert: (41, '機車', 1763, 7, '女')
insert: (41, '機車', 1763, 8, '女')
insert: (37, '機車', 1763, 9, '男')
insert: (40, '機車', 1763, 11, '男')
insert: (41, '慢車', 1763, 12, '女')
insert: (46, '機車', 1764, 1, '男')
insert: (29, '小貨車', 1764, 2, '男')
insert: (28, '機車', 1765, 1, '男')
insert: (-1, 1765, 2, '無或物')
insert: (58, '機車', 1766, 1, '女')
insert: (46, '小客車', 1766, 2, '男')
insert: (23, '曳引車', 1767, 1, '男')
insert: (82, '機車', 1767, 2, '男')
insert: (46, '小客車', 1767, 3, '男')
insert: (58, '小客車', 1768, 1, '男')
insert: (89, '慢車', 1768, 2, '男')
insert: (28, '機車', 1769, 1, '男')
insert: (61, '人', 1769, 2, '男')
insert: (46, '小貨車', 1770, 1, '男')
insert: (63, '機車', 1770, 2, '男')
insert: (70, '機車', 1771, 1, '男')
insert: (0, 1771, 2, '無或物')
insert:

4392it [00:16, 271.24it/s]

insert: (22, '機車', 1777, 2, '男')
insert: (64, '慢車', 1777, 1, '男')
insert: (72, '曳引車', 1777, 2, '男')
insert: (30, '機車', 1778, 1, '男')
insert: (58, '大貨車', 1778, 2, '男')
insert: (96, '人', 1779, 1, '男')
insert: (22, '小客車', 1779, 2, '男')
insert: (46, '大貨車', 1780, 1, '男')
insert: (45, '小貨車', 1780, 2, '男')
insert: (31, '小貨車', 1780, 3, '女')
insert: (29, '大貨車', 1780, 4, '男')
insert: (53, '半聯結車', 1780, 5, '男')
insert: (70, '小客車', 1780, 6, '男')
insert: (-1, '人', 1780, 7, '男')
insert: (56, '人', 1780, 8, '女')
insert: (28, '人', 1780, 9, '女')
insert: (28, '曳引車', 1781, 1, '男')
insert: (59, '小客車', 1781, 2, '女')


4425it [00:16, 263.61it/s]

insert: (65, '人', 1781, 3, '女')
insert: (39, '小客車', 1781, 4, '男')
insert: (54, '小貨車', 1781, 5, '男')
insert: (53, '曳引車', 1781, 6, '男')
insert: (66, '小客車', 1781, 7, '男')
insert: (70, '小客車', 1781, 8, '男')
insert: (38, '大貨車', 1781, 9, '男')
insert: (45, '機車', 1782, 1, '女')
insert: (-1, 1782, 2, '無或物')
insert: (42, '人', 1782, 3, '女')
insert: (53, '大客車', 1783, 1, '男')
insert: (32, '人', 1783, 2, '男')
insert: (30, '人', 1783, 3, '女')
insert: (1, '人', 1783, 4, '男')
insert: (40, '機車', 1784, 1, '女')
insert: (25, '小客車', 1784, 2, '男')
insert: (34, '機車', 1785, 1, '男')
insert: (111, 1785, 2, '無或物')
insert: (33, '小客車', 1786, 1, '男')
insert: (65, '機車', 1786, 2, '男')
insert: (61, '人', 1787, 1, '女')
insert: (23, '機車', 1787, 2, '男')
insert: (70, '人', 1787, 3, '男')
insert: (40, '小貨車', 1788, 1, '男')
insert: (64, '機車', 1788, 2, '男')
insert: (29, '機車', 1788, 3, '男')


In [20]:
cursor.close()

In [21]:
conn.close()